In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import safe

# Load the trained model and tokenizer
model_id = "./.saved_model/phi1_5-safmol"
tokenizer = AutoTokenizer.from_pretrained(model_id+"/checkpoint-100")
model = AutoModelForCausalLM.from_pretrained(model_id)

# Set the model to evaluation mode
model.eval()

# Define generation parameters
generation_args = {
    "max_length": 50,  # Adjust the maximum length of the generated sequence
    "num_return_sequences": 1,  # Number of sequences to generate
    "temperature": 1.0,  # Sampling temperature
    "top_k": 50,  # Top-k sampling
    "top_p": 0.95,  # Top-p (nucleus) sampling
    "do_sample": True,  # Use sampling instead of greedy decoding
}

# Generate a sequence without a prompt
input_ids = torch.tensor([[tokenizer.pad_token_id]])  # Use the pad token ID as a dummy input

with torch.no_grad():
    generated_sequences = model.generate(input_ids, **generation_args)

# Decode the generated sequence
generated_text = tokenizer.decode(generated_sequences[0], skip_special_tokens=True)

print("Generated text:", generated_text)

print("SMILES: ", safe.decode(generated_text, canonical=True) )


loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file phi1_5_updated/config.json
Model config PhiConfig {
  "_name_or_path": "phi1_5_updated",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": null,
  "embd_pdrop": 0.0,
  "eos_token_id": null,
  "hidden_act": "gelu_new",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "phi",
  "num_attention_heads": 32,
  "num_hidden_layers": 24,
  "num_key_value_heads": 32,
  "partial_rotary_factor": 0.5,
  "qk_layernorm": false,
  "resid_pdrop": 0.0,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.41.1",
  "use_cache": true,
  "vocab_size": 1880
}

loading weights file phi1_5_updated/model.safetensors
Generat

Generated text: [N5P][Th][N3[Sc+2][Th]%1B%47[Br-]2+][Br-][Br-]g+][14[Th][N3[17F][Th][N5[Ni-4][Br-]2+][Br-][Br-][N5P][Th][N3B[Br-][Br-][N4P][Th][N5[Sc+2][Th]%1


SAFEDecodeError: Failed to decode [N5P][Th][N3[Sc+2][Th]%1B%47[Br-]2+][Br-][Br-]g+][14[Th][N3[17F][Th][N5[Ni-4][Br-]2+][Br-][Br-][N5P][Th][N3B[Br-][Br-][N4P][Th][N5[Sc+2][Th]%1